# BiGRU Model for Roller Coaster Rating Prediction

This notebook trains a Bidirectional GRU model to predict roller coaster ratings from accelerometer data.

**Data Source**: Complete coaster mapping with perfect matches, duplicate averaging, and airtime calculation.

**Key Features**:
- Loads data from `complete_coaster_mapping.csv`
- Filters for perfect matches (≥95% similarity)
- Averages duplicate coaster names
- Calculates airtime features from vertical g-forces
- Trains BiGRU model with sequence + airtime features
- Full training pipeline in one notebook

In [ ]:
# ============================================================================
# IMPORTS AND CONFIGURATION
# ============================================================================

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error
import numpy as np
import pandas as pd
import glob
import os
from pathlib import Path
from collections import defaultdict
import matplotlib.pyplot as plt

# Set random seeds for reproducibility
np.random.seed(42)
torch.manual_seed(42)

# ============================================================================
# CONFIGURATION
# ============================================================================

# Data paths
MAPPING_CSV = 'ratings_data/complete_coaster_mapping.csv'

# Model hyperparameters
SEQUENCE_LENGTH = 1000
ACCEL_CHANNELS = 3
AIRTIME_FEATURE_COUNT = 4
HIDDEN_DIM = 128
NUM_LAYERS = 2
DROPOUT_RATE = 0.3
BATCH_SIZE = 16
LEARNING_RATE = 1e-4
NUM_EPOCHS = 100
WEIGHT_DECAY = 1e-5

# Data filtering
MIN_CSV_COUNT = 1
MIN_RATINGS = 10

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# ============================================================================
# DATA PREPARATION FUNCTIONS
# ============================================================================

def load_complete_mapping(csv_path=MAPPING_CSV):
    """Load the complete coaster mapping CSV."""
    df = pd.read_csv(csv_path)
    print(f"\n[OK] Loaded {len(df)} coasters from complete mapping")
    return df


def filter_perfect_matches(df):
    """Filter for perfect matches only (>=95% similarity)."""
    perfect = df[df['match_type'] == 'perfect'].copy()
    print(f"[OK] Filtered to {len(perfect)} coasters with perfect name matches")
    return perfect


def aggregate_duplicates(df):
    """Average ratings for duplicate coaster names."""
    grouped = df.groupby('coaster_name').agg({
        'avg_rating': 'mean',
        'total_ratings': 'sum',
        'csv_count': 'max',
        'full_path': 'first',
        'rfdb_park_folder': 'first',
        'rfdb_coaster_folder': 'first',
        'coaster_id': 'first',
        'ratings_coaster': 'first',
        'ratings_park': 'first',
        'match_type': 'first'
    }).reset_index()
    
    duplicates = df['coaster_name'].value_counts()
    duplicates = duplicates[duplicates > 1].index
    
    if len(duplicates) > 0:
        print(f"\n[!] Found {len(duplicates)} duplicate coaster names:")
        for coaster_name in duplicates:
            dupes = df[df['coaster_name'] == coaster_name].sort_values('csv_count', ascending=False)
            best_path = dupes.iloc[0]['full_path']
            best_csv_count = dupes.iloc[0]['csv_count']
            avg_rating = dupes['avg_rating'].mean()
            
            grouped.loc[grouped['coaster_name'] == coaster_name, 'full_path'] = best_path
            grouped.loc[grouped['coaster_name'] == coaster_name, 'csv_count'] = best_csv_count
            grouped.loc[grouped['coaster_name'] == coaster_name, 'avg_rating'] = avg_rating
            
            print(f"  - {coaster_name}: {len(dupes)} entries -> avg rating {avg_rating:.2f}")
    
    print(f"\n[OK] Final dataset: {len(grouped)} unique coasters")
    return grouped


def get_accelerometer_files(full_path):
    """Get list of accelerometer CSV files from path."""
    # Normalize path for Windows/Unix compatibility
    full_path = full_path.replace('\\', '/')
    csv_files = glob.glob(f"{full_path}/*.csv")
    return sorted(csv_files)


def load_last_accelerometer_file(full_path):
    """Load the last CSV file from the path."""
    # Normalize path separators
    full_path = full_path.replace('\\', '/')
    
    csv_files = get_accelerometer_files(full_path)
    
    if not csv_files:
        # Try checking if path exists
        if not os.path.exists(full_path):
            print(f"[!] Path does not exist: {full_path}")
        return None
    
    last_file = csv_files[-1]
    
    try:
        df = pd.read_csv(last_file)
        return df
    except Exception as e:
        print(f"Error loading {last_file}: {e}")
        return None


def calculate_airtime(accel_df, threshold=-0.1):
    """Calculate airtime statistics from accelerometer data."""
    if accel_df is None or 'Vertical' not in accel_df.columns:
        return np.zeros(AIRTIME_FEATURE_COUNT, dtype=np.float32)
    
    vertical = accel_df['Vertical'].values
    airtime_mask = vertical < threshold
    
    total_airtime_samples = np.sum(airtime_mask)
    max_negative_g = np.min(vertical) if len(vertical) > 0 else 0
    
    # Count distinct airtime moments
    airtime_moments = 0
    in_airtime = False
    for is_airtime in airtime_mask:
        if is_airtime and not in_airtime:
            airtime_moments += 1
            in_airtime = True
        elif not is_airtime:
            in_airtime = False
    
    # Calculate durations (assuming 10Hz sampling)
    sampling_rate = 10
    total_airtime = total_airtime_samples / sampling_rate
    avg_airtime_duration = total_airtime / airtime_moments if airtime_moments > 0 else 0
    
    return np.array([total_airtime, max_negative_g, airtime_moments, avg_airtime_duration], dtype=np.float32)


def prepare_training_data(mapping_csv=MAPPING_CSV, min_csv_count=MIN_CSV_COUNT, min_ratings=MIN_RATINGS):
    """Prepare complete training dataset from mapping CSV."""
    print("\n" + "="*70)
    print("PREPARING TRAINING DATA FROM COMPLETE MAPPING")
    print("="*70)
    
    df = load_complete_mapping(mapping_csv)
    df = filter_perfect_matches(df)
    
    df = df[df['csv_count'] >= min_csv_count]
    print(f"[OK] Filtered to {len(df)} coasters with >={min_csv_count} CSV files")
    
    df = df[df['total_ratings'] >= min_ratings]
    print(f"[OK] Filtered to {len(df)} coasters with >={min_ratings} total ratings")
    
    df = aggregate_duplicates(df)
    df = df.sort_values('avg_rating', ascending=False)
    
    print("\n" + "="*70)
    print("DATASET STATISTICS")
    print("="*70)
    print(f"Total coasters: {len(df)}")
    print(f"Rating range: {df['avg_rating'].min():.2f} - {df['avg_rating'].max():.2f}")
    print(f"Average rating: {df['avg_rating'].mean():.2f} +/- {df['avg_rating'].std():.2f}")
    print(f"Total CSV files: {df['csv_count'].sum()}")
    print(f"Average CSVs per coaster: {df['csv_count'].mean():.2f}")
    
    print("\nTop 5 highest rated:")
    for _, row in df.head(5).iterrows():
        print(f"  {row['coaster_name']:30s} {row['avg_rating']:.2f}* ({row['csv_count']} CSVs)")
    
    print("\nBottom 5 lowest rated:")
    for _, row in df.tail(5).iterrows():
        print(f"  {row['coaster_name']:30s} {row['avg_rating']:.2f}* ({row['csv_count']} CSVs)")
    
    return df


print("[OK] Configuration and data preparation functions loaded successfully")

Using device: cpu
[OK] Configuration and data preparation functions loaded successfully


## Step 1: Load and Prepare Data

Load the complete coaster mapping, filter for perfect matches, and aggregate duplicates.

In [2]:
# Load and prepare the dataset
coaster_mapping = prepare_training_data()

print(f"\n[OK] Ready to train on {len(coaster_mapping)} coasters")


PREPARING TRAINING DATA FROM COMPLETE MAPPING

[OK] Loaded 1299 coasters from complete mapping
[OK] Filtered to 518 coasters with perfect name matches
[OK] Filtered to 518 coasters with >=1 CSV files
[OK] Filtered to 508 coasters with >=10 total ratings

[!] Found 68 duplicate coaster names:
  - Boomerang: 13 entries -> avg rating 2.04
  - Woodstock Express: 8 entries -> avg rating 1.71
  - Dragon: 7 entries -> avg rating 2.08
  - Cobra: 7 entries -> avg rating 2.09
  - Sea Serpent: 7 entries -> avg rating 1.40
  - Corkscrew: 6 entries -> avg rating 2.08
  - Goliath: 6 entries -> avg rating 3.80
  - Tornado: 6 entries -> avg rating 2.85
  - Wildcat: 6 entries -> avg rating 2.55
  - Batman The Ride: 5 entries -> avg rating 3.69
  - Joker: 5 entries -> avg rating 3.12
  - Cyclone: 5 entries -> avg rating 2.50
  - Vortex: 5 entries -> avg rating 2.64
  - Comet: 4 entries -> avg rating 2.72
  - Viper: 4 entries -> avg rating 2.73
  - Pandemonium: 4 entries -> avg rating 2.79
  - Manta: 3 

## Step 2: Load Accelerometer Data and Extract Features

Load acceleration sequences and calculate airtime features for each coaster.

In [6]:
def process_accelerometer_data(coaster_mapping):
    """
    Process all accelerometer files and extract features.
    Returns sequences, airtime features, and ratings.
    """
    all_sequences = []
    all_airtime_features = []
    all_ratings = []
    all_coaster_names = []
    
    REQUIRED_COLUMNS = ['time', 'xforce', 'yforce', 'zforce']
    ACCEL_COLUMNS = ['xforce', 'yforce', 'zforce']

    skipped_count = 0
    processed_count = 0
    
    print("\n" + "="*70)
    print("PROCESSING ACCELEROMETER DATA")
    print("="*70)
    
    # Check first path to verify directory structure
    if len(coaster_mapping) > 0:
        first_path = coaster_mapping.iloc[0]['full_path']
        print(f"\n[DEBUG] First path in mapping: {first_path}")
        print(f"[DEBUG] Path exists: {os.path.exists(first_path)}")
        if os.path.exists(first_path):
            files = glob.glob(f"{first_path.replace(chr(92), '/')}/*.csv")
            print(f"[DEBUG] CSV files found: {len(files)}")
            if files:
                print(f"[DEBUG] First file: {files[0]}")
    
    for idx, row in coaster_mapping.iterrows():
        coaster_name = row['coaster_name']
        full_path = row['full_path']
        rating = row['avg_rating']
        
        # Load accelerometer data
        accel_df = load_last_accelerometer_file(full_path)
        
        if accel_df is None:
            skipped_count += 1
            if skipped_count <= 3:  # Show first 3 failures for debugging
                print(f"[!] Failed to load: {coaster_name} at {full_path}")
            continue
        
        # Check for required columns
        missing_cols = [col for col in REQUIRED_COLUMNS if col not in accel_df.columns]
        if missing_cols:
            skipped_count += 1
            if skipped_count <= 3:
                print(f"[!] Missing columns for {coaster_name}: {missing_cols}")
                print(f"    Available columns: {list(accel_df.columns)}")
            continue
        
        # Convert to numeric and drop NaN
        for col in REQUIRED_COLUMNS:
            accel_df[col] = pd.to_numeric(accel_df[col], errors='coerce')
        accel_df = accel_df.dropna(subset=REQUIRED_COLUMNS)
        
        if len(accel_df) < SEQUENCE_LENGTH:
            skipped_count += 1
            continue
        
        # Calculate airtime features
        airtime_features = calculate_airtime(accel_df)
        
        # Extract acceleration sequences
        data = accel_df[ACCEL_COLUMNS].values.T  # Shape: (3, num_timesteps)
        
        # Normalize (subtract 1g from vertical, divide all by 5g)
        data[1, :] = data[1, :] - 1.0  # Vertical index is 1
        data = data / 5.0
        
        # Create sequences with overlapping windows
        num_timesteps = data.shape[1]
        step_size = SEQUENCE_LENGTH // 4
        
        for start in range(0, num_timesteps - SEQUENCE_LENGTH + 1, step_size):
            segment = data[:, start:start + SEQUENCE_LENGTH]
            
            if segment.shape[1] == SEQUENCE_LENGTH:
                all_sequences.append(segment)
                all_airtime_features.append(airtime_features)
                all_ratings.append(rating)
                all_coaster_names.append(coaster_name)
        
        processed_count += 1
        if processed_count % 50 == 0:
            print(f"  Processed {processed_count}/{len(coaster_mapping)} coasters...")
    
    print(f"\n[OK] Successfully processed {processed_count} coasters")
    print(f"[!] Skipped {skipped_count} coasters (missing/invalid data)")
    print(f"[OK] Generated {len(all_sequences)} training sequences")
    
    # Convert to numpy arrays
    sequences = np.array(all_sequences, dtype=np.float32)  # (N, 3, seq_len)
    airtime_features = np.array(all_airtime_features, dtype=np.float32)  # (N, 4)
    ratings = np.array(all_ratings, dtype=np.float32)  # (N,)
    
    return sequences, airtime_features, ratings, all_coaster_names


# Process all coasters
sequences, airtime_features, ratings, coaster_names = process_accelerometer_data(coaster_mapping)

print(f"\nFinal dataset shape:")
print(f"  Sequences: {sequences.shape}")
print(f"  Airtime features: {airtime_features.shape}")
print(f"  Ratings: {ratings.shape}")


PROCESSING ACCELEROMETER DATA

[DEBUG] First path in mapping: rfdb_csvs\energylandia\zadra
[DEBUG] Path exists: True
[DEBUG] CSV files found: 1
[DEBUG] First file: rfdb_csvs/energylandia/zadra\1734190639_3184.csv
  Processed 50/359 coasters...
  Processed 100/359 coasters...
  Processed 150/359 coasters...
  Processed 100/359 coasters...
  Processed 150/359 coasters...
  Processed 200/359 coasters...
  Processed 200/359 coasters...
  Processed 250/359 coasters...
  Processed 300/359 coasters...
  Processed 250/359 coasters...
  Processed 300/359 coasters...
  Processed 350/359 coasters...

[OK] Successfully processed 354 coasters
[!] Skipped 5 coasters (missing/invalid data)
[OK] Generated 3355 training sequences

Final dataset shape:
  Sequences: (3355, 3, 1000)
  Airtime features: (3355, 4)
  Ratings: (3355,)
  Processed 350/359 coasters...

[OK] Successfully processed 354 coasters
[!] Skipped 5 coasters (missing/invalid data)
[OK] Generated 3355 training sequences

Final dataset sh

## Step 3: Create Dataset and DataLoaders

Split data into train/val/test sets and create PyTorch DataLoaders.

In [7]:
class CoasterDataset(Dataset):
    """PyTorch dataset for coaster sequences and airtime features."""
    
    def __init__(self, sequences, airtime_features, ratings):
        self.sequences = torch.from_numpy(sequences).float()
        self.airtime_features = torch.from_numpy(airtime_features).float()
        self.ratings = torch.from_numpy(ratings).float()
    
    def __len__(self):
        return len(self.sequences)
    
    def __getitem__(self, idx):
        return self.sequences[idx], self.airtime_features[idx], self.ratings[idx]


# Normalize features
scaler_airtime = StandardScaler()
scaler_ratings = StandardScaler()

airtime_features_normalized = scaler_airtime.fit_transform(airtime_features)
ratings_normalized = scaler_ratings.fit_transform(ratings.reshape(-1, 1)).flatten()

# Split data: 70% train, 15% val, 15% test
indices = np.arange(len(sequences))
train_idx, temp_idx = train_test_split(indices, test_size=0.3, random_state=42)
val_idx, test_idx = train_test_split(temp_idx, test_size=0.5, random_state=42)

# Create datasets
train_dataset = CoasterDataset(
    sequences[train_idx],
    airtime_features_normalized[train_idx],
    ratings_normalized[train_idx]
)

val_dataset = CoasterDataset(
    sequences[val_idx],
    airtime_features_normalized[val_idx],
    ratings_normalized[val_idx]
)

test_dataset = CoasterDataset(
    sequences[test_idx],
    airtime_features_normalized[test_idx],
    ratings_normalized[test_idx]
)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

print("\n" + "="*70)
print("DATASET SPLIT")
print("="*70)
print(f"Train sequences: {len(train_dataset)}")
print(f"Val sequences: {len(val_dataset)}")
print(f"Test sequences: {len(test_dataset)}")
print(f"Batch size: {BATCH_SIZE}")
print(f"Train batches: {len(train_loader)}")
print(f"Val batches: {len(val_loader)}")
print(f"Test batches: {len(test_loader)}")


DATASET SPLIT
Train sequences: 2348
Val sequences: 503
Test sequences: 504
Batch size: 16
Train batches: 147
Val batches: 32
Test batches: 32


## Step 4: Define BiGRU Model

Bidirectional GRU with airtime feature integration.

In [8]:
class BiGRURegressor(nn.Module):
    """
    Bidirectional GRU model for coaster rating prediction.
    Combines accelerometer sequences with airtime features.
    """
    
    def __init__(self, accel_input_size=ACCEL_CHANNELS, airtime_feature_size=AIRTIME_FEATURE_COUNT,
                 hidden_dim=HIDDEN_DIM, num_layers=NUM_LAYERS, dropout_rate=DROPOUT_RATE):
        super(BiGRURegressor, self).__init__()
        
        # BiGRU for sequence processing
        self.gru = nn.GRU(
            input_size=accel_input_size,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True,
            bidirectional=True,
            dropout=dropout_rate if num_layers > 1 else 0
        )
        
        # Process airtime features
        self.airtime_head = nn.Sequential(
            nn.Linear(airtime_feature_size, hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout_rate / 2)
        )
        
        # Combined embedding size
        final_input_size = (2 * hidden_dim) + hidden_dim
        
        # Regression head
        self.head = nn.Sequential(
            nn.Linear(final_input_size, hidden_dim * 2),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(hidden_dim * 2, hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(hidden_dim, 1)
        )
    
    def forward(self, x_accel, x_airtime):
        # Process acceleration sequence
        x_accel = x_accel.transpose(1, 2)  # (batch, channels, seq_len) -> (batch, seq_len, channels)
        _, h_n = self.gru(x_accel)
        
        # Concatenate forward and backward hidden states
        rnn_embedding = torch.cat((h_n[-2, :, :], h_n[-1, :, :]), dim=1)
        
        # Process airtime features
        airtime_embedding = self.airtime_head(x_airtime)
        
        # Combine embeddings
        combined_embedding = torch.cat((rnn_embedding, airtime_embedding), dim=1)
        
        # Final prediction
        output = self.head(combined_embedding).squeeze(1)
        
        return output


# Create model
model = BiGRURegressor()
model = model.to(device)

# Count parameters
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print("\n" + "="*70)
print("MODEL ARCHITECTURE")
print("="*70)
print(model)
print(f"\nTotal parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")


MODEL ARCHITECTURE
BiGRURegressor(
  (gru): GRU(3, 128, num_layers=2, batch_first=True, dropout=0.3, bidirectional=True)
  (airtime_head): Sequential(
    (0): Linear(in_features=4, out_features=128, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.15, inplace=False)
  )
  (head): Sequential(
    (0): Linear(in_features=384, out_features=256, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.3, inplace=False)
    (3): Linear(in_features=256, out_features=128, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.3, inplace=False)
    (6): Linear(in_features=128, out_features=1, bias=True)
  )
)

Total parameters: 530,817
Trainable parameters: 530,817


## Step 5: Training Loop

Train the model with early stopping and validation monitoring.

In [13]:
# Setup optimizer and loss
optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
criterion = nn.MSELoss()

# Learning rate scheduler (removed verbose parameter - deprecated in newer PyTorch versions)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5)

# Training history
history = {
    'train_loss': [],
    'val_loss': [],
    'val_r2': [],
    'val_mse': []
}

best_val_loss = float('inf')
patience_counter = 0
PATIENCE = 15

print("\n" + "="*70)
print("TRAINING")
print("="*70)

for epoch in range(NUM_EPOCHS):
    # Training phase
    model.train()
    train_loss = 0.0
    
    for sequences, airtime_feats, ratings in train_loader:
        sequences = sequences.to(device)
        airtime_feats = airtime_feats.to(device)
        ratings = ratings.to(device)
        
        optimizer.zero_grad()
        predictions = model(sequences, airtime_feats)
        loss = criterion(predictions, ratings)
        loss.backward()
        
        # Gradient clipping
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        
        optimizer.step()
        train_loss += loss.item()
    
    avg_train_loss = train_loss / len(train_loader)
    
    # Validation phase
    model.eval()
    val_loss = 0.0
    all_val_preds = []
    all_val_targets = []
    
    with torch.no_grad():
        for sequences, airtime_feats, ratings in val_loader:
            sequences = sequences.to(device)
            airtime_feats = airtime_feats.to(device)
            ratings = ratings.to(device)
            
            predictions = model(sequences, airtime_feats)
            loss = criterion(predictions, ratings)
            val_loss += loss.item()
            
            all_val_preds.extend(predictions.cpu().numpy())
            all_val_targets.extend(ratings.cpu().numpy())
    
    avg_val_loss = val_loss / len(val_loader)
    
    # Calculate metrics
    val_r2 = r2_score(all_val_targets, all_val_preds)
    val_mse = mean_squared_error(all_val_targets, all_val_preds)
    
    # Update history
    history['train_loss'].append(avg_train_loss)
    history['val_loss'].append(avg_val_loss)
    history['val_r2'].append(val_r2)
    history['val_mse'].append(val_mse)
    
    # Learning rate scheduling
    old_lr = optimizer.param_groups[0]['lr']
    scheduler.step(avg_val_loss)
    new_lr = optimizer.param_groups[0]['lr']
    
    # Early stopping
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        patience_counter = 0
        torch.save(model.state_dict(), 'best_bigru_model.pth')
        best_marker = " *"
    else:
        patience_counter += 1
        best_marker = ""
    
    # Print progress for every epoch
    print(f"Epoch {epoch+1:3d}/{NUM_EPOCHS} | "
          f"Train: {avg_train_loss:.4f} | "
          f"Val: {avg_val_loss:.4f} | "
          f"R2: {val_r2:.4f}{best_marker}")
    
    # Manually print LR change (since verbose was removed)
    if new_lr != old_lr:
        print(f"  -> Learning rate reduced: {old_lr:.6f} -> {new_lr:.6f}")
    
    # Early stopping check
    if patience_counter >= PATIENCE:
        print(f"\nEarly stopping triggered at epoch {epoch+1}")
        break

print("\n[OK] Training completed!")
print(f"Best validation loss: {best_val_loss:.4f}")


TRAINING


KeyboardInterrupt: 

## Step 6: Visualize Training History

Plot training and validation losses over epochs.

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot losses
axes[0].plot(history['train_loss'], label='Train Loss', linewidth=2)
axes[0].plot(history['val_loss'], label='Val Loss', linewidth=2)
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss (MSE)')
axes[0].set_title('Training and Validation Loss')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Plot R2 score
axes[1].plot(history['val_r2'], label='Val R2', color='green', linewidth=2)
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('R² Score')
axes[1].set_title('Validation R² Score')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nFinal Training Loss: {history['train_loss'][-1]:.4f}")
print(f"Final Validation Loss: {history['val_loss'][-1]:.4f}")
print(f"Final Validation R²: {history['val_r2'][-1]:.4f}")

## Step 7: Evaluate on Test Set

Load best model and evaluate on held-out test data.

In [ ]:
# Load best model
model.load_state_dict(torch.load('best_bigru_model.pth'))
model.eval()

# Evaluate on test set
test_loss = 0.0
all_test_preds = []
all_test_targets = []

print("\n" + "="*70)
print("TEST SET EVALUATION")
print("="*70)

with torch.no_grad():
    for sequences, airtime_feats, ratings in test_loader:
        sequences = sequences.to(device)
        airtime_feats = airtime_feats.to(device)
        ratings = ratings.to(device)
        
        predictions = model(sequences, airtime_feats)
        loss = criterion(predictions, ratings)
        test_loss += loss.item()
        
        all_test_preds.extend(predictions.cpu().numpy())
        all_test_targets.extend(ratings.cpu().numpy())

avg_test_loss = test_loss / len(test_loader)
test_r2 = r2_score(all_test_targets, all_test_preds)
test_mse = mean_squared_error(all_test_targets, all_test_preds)
test_rmse = np.sqrt(test_mse)

# Denormalize predictions for interpretability
all_test_preds_denorm = scaler_ratings.inverse_transform(np.array(all_test_preds).reshape(-1, 1)).flatten()
all_test_targets_denorm = scaler_ratings.inverse_transform(np.array(all_test_targets).reshape(-1, 1)).flatten()

# Calculate metrics on original scale
test_r2_denorm = r2_score(all_test_targets_denorm, all_test_preds_denorm)
test_mse_denorm = mean_squared_error(all_test_targets_denorm, all_test_preds_denorm)
test_rmse_denorm = np.sqrt(test_mse_denorm)

print(f"\nNormalized Metrics:")
print(f"  Test Loss: {avg_test_loss:.4f}")
print(f"  Test R²: {test_r2:.4f}")
print(f"  Test MSE: {test_mse:.4f}")
print(f"  Test RMSE: {test_rmse:.4f}")

print(f"\nOriginal Scale Metrics:")
print(f"  Test R²: {test_r2_denorm:.4f}")
print(f"  Test MSE: {test_mse_denorm:.4f}")
print(f"  Test RMSE: {test_rmse_denorm:.4f}")
print(f"  Mean Absolute Error: {np.mean(np.abs(all_test_targets_denorm - all_test_preds_denorm)):.4f}")

## Step 8: Prediction vs Actual Visualization

Compare model predictions with actual ratings on the test set.

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Scatter plot
axes[0].scatter(all_test_targets_denorm, all_test_preds_denorm, alpha=0.5, s=20)
axes[0].plot([min(all_test_targets_denorm), max(all_test_targets_denorm)],
             [min(all_test_targets_denorm), max(all_test_targets_denorm)],
             'r--', linewidth=2, label='Perfect Prediction')
axes[0].set_xlabel('Actual Rating')
axes[0].set_ylabel('Predicted Rating')
axes[0].set_title(f'Predictions vs Actual (R² = {test_r2_denorm:.3f})')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Residual plot
residuals = all_test_targets_denorm - all_test_preds_denorm
axes[1].scatter(all_test_preds_denorm, residuals, alpha=0.5, s=20)
axes[1].axhline(y=0, color='r', linestyle='--', linewidth=2)
axes[1].set_xlabel('Predicted Rating')
axes[1].set_ylabel('Residual (Actual - Predicted)')
axes[1].set_title('Residual Plot')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Show some example predictions
print("\n" + "="*70)
print("SAMPLE PREDICTIONS")
print("="*70)
sample_indices = np.random.choice(len(all_test_targets_denorm), min(10, len(all_test_targets_denorm)), replace=False)
for idx in sample_indices:
    actual = all_test_targets_denorm[idx]
    predicted = all_test_preds_denorm[idx]
    error = abs(actual - predicted)
    print(f"Actual: {actual:.2f} | Predicted: {predicted:.2f} | Error: {error:.2f}")

## Step 9: Save Model and Scalers

Save the trained model and scalers for future use.

In [ ]:
import pickle

# Save model
torch.save({
    'model_state_dict': model.state_dict(),
    'model_config': {
        'accel_input_size': ACCEL_CHANNELS,
        'airtime_feature_size': AIRTIME_FEATURE_COUNT,
        'hidden_dim': HIDDEN_DIM,
        'num_layers': NUM_LAYERS,
        'dropout_rate': DROPOUT_RATE
    },
    'train_config': {
        'sequence_length': SEQUENCE_LENGTH,
        'batch_size': BATCH_SIZE,
        'learning_rate': LEARNING_RATE,
        'num_epochs': NUM_EPOCHS
    }
}, 'bigru_rating_model.pth')

# Save scalers
with open('scaler_airtime.pkl', 'wb') as f:
    pickle.dump(scaler_airtime, f)

with open('scaler_ratings.pkl', 'wb') as f:
    pickle.dump(scaler_ratings, f)

print("\n" + "="*70)
print("MODEL SAVED")
print("="*70)
print("Files saved:")
print("  - bigru_rating_model.pth (model weights + config)")
print("  - scaler_airtime.pkl (airtime feature scaler)")
print("  - scaler_ratings.pkl (rating scaler)")
print("\nTo load the model:")
print("  checkpoint = torch.load('bigru_rating_model.pth')")
print("  model = BiGRURegressor(**checkpoint['model_config'])")
print("  model.load_state_dict(checkpoint['model_state_dict'])")

## Summary

This notebook provides a complete pipeline for:

1. **Data Loading**: Load coaster mapping from `complete_coaster_mapping.csv`
2. **Data Filtering**: Filter for perfect matches (≥95% similarity)
3. **Duplicate Handling**: Average ratings for duplicate coaster names
4. **Feature Extraction**: 
   - Accelerometer sequences (3-axis: Lateral, Vertical, Longitudinal)
   - Airtime features (total airtime, max negative g, airtime moments, avg duration)
5. **Model Training**: Bidirectional GRU with airtime feature integration
6. **Evaluation**: R², MSE, RMSE on test set
7. **Model Saving**: Save trained model and scalers for deployment

**Key Results:**
- Dataset: 359 unique coasters after filtering and aggregation
- Rating range: 1.06 - 4.90 stars
- Model: BiGRU with ~{trainable_params:,} trainable parameters
- Test R²: {test_r2_denorm:.4f}
- Test RMSE: {test_rmse_denorm:.4f} stars

The trained model can now be used in production for rating prediction from accelerometer data!